# Homework 2: Perfect Numbers Solutions
#### Author: Maurik Holtrop (2018)
This was a challenger. The code I present here works reasonably well. It can probably go a little faster if tweaked.

### Task: Proper Divisors
We need these to check the perfect number code. A proper divisor is an integer that will divide a larger integer without remainder.

In [1]:
import math
import numpy as np

def divisorList(N):
    '''Quick divisor list creator. Returns a list of perfect devisor to input N'''
    divs=[i for i in range(1,int(math.sqrt(N)+1)) if N%i==0 ]  # Only do up to sqrt(N)
    divs+=reversed([ int(N/i) for i in divs] )                 # The rest are the N/i 
    return(divs)
#
# This implements the algorithm as a "generator" (same as range() in Python3 is a generator, or xrange() in P2)
# It is pretty fast, but not faster than our simple list creator. For very large n, this one
# would become interesting because it uses half the memory.
def divisorGenerator(n):
    '''This is needed to test directly from a perfect number, the brute force way.
    This function returns a list of perfect divisors for a number n.
    It is pretty fast, since it only brute force checks up to sqrt(n), and then lists the 
    complements. 
    Since this is a generator, it needs to be in a loop or a list()'''
    large_divisors = []
    for i in range(1, int(math.sqrt(n) + 1)):
        if n % i == 0:
            yield i # Return the i
            if i*i != n:
                large_divisors.append(int(n / i)) # Store the > sqrt(N) ones for later.
    for divisor in reversed(large_divisors):
        yield divisor

**Always test your code!**

In [2]:
N=12313420
%time div1 = divisorList(N)
%time div2 = list(divisorGenerator(N))
print(len(div1),len(div2))
print("Comparing lists:")
for i in range(len(div2)):
    if div1[i]!=div2[i]:
        print("Problem at i:",i,div1[i],div2[i])

               

CPU times: user 318 µs, sys: 5 µs, total: 323 µs
Wall time: 336 µs
CPU times: user 348 µs, sys: 5 µs, total: 353 µs
Wall time: 368 µs
48 48
Comparing lists:


## Perfect numbers part 1
Here we have a simple, straight forward perfect number finder, It is too slow for a 10^6 number, so forget doing 10^10 which is what we need for Google's number. It is still useful to check other algorithms against.

In [3]:
def find_perfect_number(Max=1000000,Min=1):
    '''This is the slow brute force method to find perfect numbers. It is OK up to about 10**5
    but gets too slow for larger numbers.'''
    for ntry in range(Min,Max):
        pp=list(divisorGenerator(ntry))
        s= np.sum(pp[0:-1])
        if s == ntry:
            print("Perfect number {} from ".format(ntry),pp[0:-1])


In [4]:
print("This will take a bit more than 2 seconds...")
%time(find_perfect_number(100000))

This will take a bit more than 2 seconds...
Perfect number 6 from  [1, 2, 3]
Perfect number 28 from  [1, 2, 4, 7, 14]
Perfect number 496 from  [1, 2, 4, 8, 16, 31, 62, 124, 248]
Perfect number 8128 from  [1, 2, 4, 8, 16, 32, 64, 127, 254, 508, 1016, 2032, 4064]
CPU times: user 2.1 s, sys: 7.55 ms, total: 2.11 s
Wall time: 2.11 s


## Perfect numbers part 2: Euclids Method.
Here is what Euclid stated, well, translated to English:
> If as many numbers as we please beginning from a unit [1] be set out continuously in double proportion, 
> until the sum of all becomes a prime, and if the sum multiplied into the last make some number, the product 
> will be perfect.

OK, so we go 1+2+4 = 7, which is a prime, so 4\*7=28 is a perfect number.

We can calcuate this quicker by realizing that 1+2+4 is equal to 2\*\*3-1, and 4 is 2\*\*2, so a perfect number is
found as (2\*\*(N-1))\*(2\*\*N -1), if (2\*\*N -1) is prime. Such primes are known as Mersenne primes. 
The largest currently known Mersenne prime is 2\*\*77232917 -1. If you want you *can* use Python to count the digits.

In [5]:
#xx=77232917
xx=77232917
NN=2**xx -1
# print("Warning: This takes a long time to compute!!!")
# print(len(str(NN)))
print(int(math.log10(NN))+1)
print(int(xx*math.log10(2))+1)


23249425
23249425


Here is the Euclids method function. It makes use of the fast Prime Sieve of HW2 part 2.

In [11]:
def is_prime(n):
    '''This is a simple prime number checker. Return true if n is a prime number.'''
    if n < 2:
        return(False)
    if n==2:
        return(True)
    if n%2 == 0:
        return(False)
    else:
        for x in range(3,int(n**0.5)+1,2):  # No need to test more than sqrt of n, turned integer, rounded up. Only odds
            if n%x == 0:
                return(False)
    return(True)

def primeSieve(sieveSize):
    ''' Returns a list of prime numbers, up to sieveSize, calculated using
    the Sieve of Eratosthenes algorithm. Not practical for very
    large sieveSize (too slow, and too much memory). 2**24 takes about 2 seconds.
    This version is more than twice the speed of the previous (in comments)''' 
    # The following line does the same as the 5 lines above. Note how it treats sieveSize odd and even.
    sieve = [False,False,True,True]+[False,True]*(sieveSize//2 -2) + [False]*(sieveSize%2!=0)
    # create the sieve
    for i in range(3, int(math.sqrt(sieveSize)) + 1,2): # You only need to sieve every other up to sqrt(N)
        if sieve[i] == True:                          # Only need to check numbers that still may be primes.
#            for i in range(i*i,sieveSize,i):    # Mark every i-th value false, starting at i*i 
#                sieve[i]=False
#  This trick uses "slicing" of the list to quickly set particular values to False
            sieve[i*i::i]=[False]*((len(sieve)-i*i+i-1)//i) # len(sieve[i*i::i])
# compile the list of primes and return
    return [ i for i in range(sieveSize) if sieve[i]==True]  # This list comprehension version is faster.



In [12]:
test_prime=2**31-1
print(test_prime)
%time print(is_prime(test_prime))
%time primes=primeSieve(2**20)

2147483647
True
CPU times: user 3.35 ms, sys: 166 µs, total: 3.51 ms
Wall time: 3.45 ms
CPU times: user 118 ms, sys: 17.5 ms, total: 136 ms
Wall time: 135 ms


In [17]:
def find_perfect_number_euclid(Max=60,Min=1):
    ''' Use Euclids method to find perfect numbers up to 2**Max, it is OK up to about Max=1000
    Most of the time is spend finding the Mersenne primes.'''
    perfects=[]
    for ntry in range(Min,Max):
        # Using is_prime3, we switch to a very large prime number tester that is not 100% accurate!
        if is_prime(2**ntry-1):
            perfects.append((ntry,(2**ntry-1)*2**(ntry-1)))
    return(perfects)


In [19]:
print("This works reasonable up to 2**60. Too slow after that.")
%time perfects=find_perfect_number_euclid(60)
for x,p in perfects:
    print("[{:3d}] {}".format(x,p))

This works reasonable up to 2**60. Too slow after that.
CPU times: user 14.2 ms, sys: 356 µs, total: 14.5 ms
Wall time: 14.6 ms
[  2] 6
[  3] 28
[  5] 496
[  7] 8128
[ 13] 33550336
[ 17] 8589869056
[ 19] 137438691328
[ 31] 2305843008139952128


We got the Google number though, it is: $2^{16}(2^{17}-1) = 8589869056$ 

### Perfect Numbers part 3: Really Big Now
The "problem" with our last attempt is that the prime number finder is too slow for numbers larger than about 2\*\*60. We need something a lot more sophisticated. There is a complicated algorithm due to Rabin-Miller. This is the kind of algorithm you look up instead of coding it yourself.  

In [41]:
import random
def rabinMiller(num):
    '''Returns True if num is a probably a prime number. Since it uses trial
    functions, there is no guarantee. It is quite good though, and usually 
    the number is indeed prime if this returns true.'''
    #
    # This is not my code. I found this simplified algorithm at: https://inventwithpython.com/rabinMiller.py 
    # There are more extended codes available at: https://rosettacode.org/wiki/Miller–Rabin_primality_test
    #
    if num==2 or num==3: 
        return(True)
    if num==0 or num==1 or num%2==0:
        return(False)
    
    s = num - 1
    t = 0
    while s % 2 == 0:
        # keep halving s while it is even (and use t
        # to count how many times we halve s)
        s = s // 2
        t += 1

    for trials in range(5): # try to falsify num's primality 5 times
        a = random.randrange(2, num - 1)
        v = pow(a, s, num)
        if v != 1: # this test does not apply if v is 1.
            i = 0
            while v != (num - 1):
                if i == t - 1:
                    return False
                else:
                    i = i + 1
                    v = (v ** 2) % num
    return True


def find_perfect_number_euclid_RabinMiller(Max=100,Min=1):
    ''' Use Euclids method to find perfect numbers up to 2**Max. This version uses the Rabin-Miller
    function to test for primes.'''
    perfects=[]
    for ntry in range(Min,Max):
        # Using is_prime3, we switch to a very large prime number tester that is not 100% accurate!
        if ntry<3:  # The RB does not work for numbers smaller than 2**3-1
            if is_prime(2**ntry-1):
#                print("Perfect number {} from {} doublings".format((2**ntry-1)*2**(ntry-1),ntry))
                perfects.append((ntry,(2**ntry-1)*2**(ntry-1)))
        else: 
            if rabinMiller(2**ntry-1):
#               print("Perfect number {} from {} doublings".format((2**ntry-1)*2**(ntry-1),ntry))
                perfects.append((ntry,(2**ntry-1)*2**(ntry-1)))
    return(perfects)


In [48]:
# Test the rabinMiller against the is_prime
print("This is prime: ",rabinMiller(1234567890234941))
for n in range(100000):
    if (is_prime(n)) != (rabinMiller(n)):
        print("OOPS, we have a problem at: ",n)

This is prime:  True


In [50]:
%time perfects = find_perfect_number_euclid_RabinMiller(1000)
print("All Perfect Number < 2^1000")
for i,p in perfects:
    print("[{:4d}] - {}".format(i,p))

CPU times: user 1.15 s, sys: 3.54 ms, total: 1.15 s
Wall time: 1.15 s
All Perfect Number < 2^1000
[   2] - 6
[   3] - 28
[   5] - 496
[   7] - 8128
[  13] - 33550336
[  17] - 8589869056
[  19] - 137438691328
[  31] - 2305843008139952128
[  61] - 2658455991569831744654692615953842176
[  89] - 191561942608236107294793378084303638130997321548169216
[ 107] - 13164036458569648337239753460458722910223472318386943117783728128
[ 127] - 14474011154664524427946373126085988481573677491474835889066354349131199152128
[ 521] - 23562723457267347065789548996709904988477547858392600710143027597506337283178622239730365539602600561360255566462503270175052892578043215543382498428777152427010394496918664028644534128033831439790236838624033171435922356643219703101720713163527487298747400647801939587165936401087419375649057918549492160555646976
[ 607] - 1410537837067120690632079580860631898814867435147156678388386759999548677426523801141041933290376902515619505687098293271640877243663700871167312681593136524